In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [18]:
VGG16=[64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M']
class VGG_Net(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(VGG_Net, self).__init__()
        self.in_channels=in_channels
        self.conv_layers=self.create_conv_layer(VGG16)
        self.fully_connected_layers=nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,num_classes)
        )
    def forward(self,x):
        x=self.conv_layers(x)
        print(x.shape)
        x=x.reshape(x.shape[0],-1)
        print(f"X in Forward after reshape:{x} ")
        x=self.fully_connected_layers(x)
        print(f"In forward layer: {x}")
        return x
    def create_conv_layer(self, architecture):
        layers=[]
        in_channels=self.in_channels
        
        for x in architecture:
            if type(x)==int:
                out_channels=x
                
                layers +=[nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                         nn.BatchNorm2d(x),
                         nn.ReLU()]
                in_channels=x
            elif x=='M':
                layers +=[nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
        print(f"Creating Conv_layers: {layers} \n")
        return nn.Sequential(*layers)
model=VGG_Net(in_channels=3, num_classes=1000)
x=torch.randn(1,3,224,224)
print(model)
print(model(x).shape)

Creating Conv_layers: [Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(), MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False), Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(), MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False), Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(), Conv2d(256, 256, k